<a href="https://colab.research.google.com/github/Yoonnooi/Test/blob/main/notebooks/1_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec Implementation from Scratch

This notebook demonstrates how to implement the Word2Vec algorithm from scratch using PyTorch. We'll use the first Harry Potter book as our corpus to train word embeddings.


## 1. Setting Up the Environment

First, we need to import the necessary libraries:
- `torch` and `torch.nn` for tensor operations and neural network functionality
- `string` for string manipulations (removing punctuation)


In [1]:
import torch
import torch.nn as nn
import string


## 2. Getting the Text Data

We'll download the first Harry Potter book to use as our corpus.

In [2]:
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"


--2025-03-11 04:59:19--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439742 (429K) [text/plain]
Saving to: ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’

J. K. Rowling - Har 100%[===================>] 429.44K  --.-KB/s    in 0.04s   

2025-03-11 04:59:19 (9.33 MB/s) - ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’ saved [439742/439742]



## 3. Text Preprocessing

Before we can use the text data, we need to preprocess it:
- Remove punctuation
- Convert text to lowercase
- Split text into tokens (words)

This function will help us clean and tokenize the text.

In [3]:
def remove_punctuation(x):
  return x.translate(''.maketrans('', '', string.punctuation))

def make_tokenized_corpus(corpus):
  out= [ [y.lower() for y in remove_punctuation(sentence).split(' ') if y] for sentence in corpus]
  return [x for x in out if x!=[]]


## 4. Loading and Formatting the Text

Now we'll load the text file, replace some special characters, and split the text into sentences.


In [8]:
with open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", 'r') as f:
  strings = f.readlines()
sample_text = "".join(strings).replace('\n', ' ').replace('Mr.', 'mr').replace('Mrs.', 'mrs').split('. ')
list_of_sentences = sample_text

#sample_text #list of strings(sentence)

Let's tokenize the text using our preprocessing function `make_tokenized_corpus`:

In [11]:
# Corpus is a list of list of strings (words)
#i = 0
#for sentence in list_of_sentences:
#  i += 1
#  if i > 10:
#    break
#  print(sentence)

for sentence in list_of_sentences[:10]:
  print(sentence)

Harry Potter and the Sorcerer's Stone   CHAPTER ONE  THE BOY WHO LIVED  mr and mrs Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much
They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense
 mr Dursley was the director of a firm called Grunnings, which made drills
He was a big, beefy man with hardly any neck, although he did have a very large mustache
mrs Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors
The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere
 The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it
They didn't think they could bear it if anyone found out about the Potters
mrs Potter was

In [18]:
corpus = make_tokenized_corpus(list_of_sentences[:10]) #list of list of words
type(corpus), type(corpus[0]), type(corpus[0][0])

(list, list, str)

## 5. Creating Context Word Pairs

A key concept in Word2Vec is learning from context. We need to create pairs of words that appear near each other in the text. We'll use a sliding window approach to create these pairs.

For example, with the window size of 2, for the word "to" in the sentence "they were the last people youd expect to be involved...", we would create pairs with:
- ("to", "expect")
- ("to", "be")
- ("to", "involved")
- ("to", "in")

These pairs will be our training data.

In [35]:
from tqdm import tqdm

sample_sentence = ['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']
idx = 4
window_size = 2
center_word = sample_sentence[idx]
word_pairs = []

for sample_sentence in corpus:
  for idx in range(len(sample_sentence)):
    center_word = sample_sentence[idx]
    context_start = max(idx - window_size, 0)
    context_end = min(idx + window_size + 1, len(sample_sentence))
    for i in range(context_start, context_end):
      if (i == idx):
        continue
      word_pair = (center_word, sample_sentence[i])
      word_pairs.append(word_pair)
print(word_pairs)

[('harry', 'potter'), ('harry', 'and'), ('potter', 'harry'), ('potter', 'and'), ('potter', 'the'), ('and', 'harry'), ('and', 'potter'), ('and', 'the'), ('and', 'sorcerers'), ('the', 'potter'), ('the', 'and'), ('the', 'sorcerers'), ('the', 'stone'), ('sorcerers', 'and'), ('sorcerers', 'the'), ('sorcerers', 'stone'), ('sorcerers', 'chapter'), ('stone', 'the'), ('stone', 'sorcerers'), ('stone', 'chapter'), ('stone', 'one'), ('chapter', 'sorcerers'), ('chapter', 'stone'), ('chapter', 'one'), ('chapter', 'the'), ('one', 'stone'), ('one', 'chapter'), ('one', 'the'), ('one', 'boy'), ('the', 'chapter'), ('the', 'one'), ('the', 'boy'), ('the', 'who'), ('boy', 'one'), ('boy', 'the'), ('boy', 'who'), ('boy', 'lived'), ('who', 'the'), ('who', 'boy'), ('who', 'lived'), ('who', 'mr'), ('lived', 'boy'), ('lived', 'who'), ('lived', 'mr'), ('lived', 'and'), ('mr', 'who'), ('mr', 'lived'), ('mr', 'and'), ('mr', 'mrs'), ('and', 'lived'), ('and', 'mr'), ('and', 'mrs'), ('and', 'dursley'), ('mrs', 'mr'), (

## 6. Building the Vocabulary

To work with word vectors, we need to create a vocabulary that maps each unique word to an index. We'll also filter out rare words that appear less than a certain number of times in the corpus.

### 6.1 Collecting All Words

First, let's collect all words in our corpus:


In [ ]:
# we have to make vocabulary




### 6.2 Finding Unique Words

Now, let's find the unique words in our corpus:

In [ ]:
# we have to get the "unique" item among total words

### 6.3 Converting to a List and Sorting

We'll convert the set of unique words to a sorted list:

In [ ]:
# vocab_set[0] # set is not subscriptable because it has no order

### 6.4 Filtering by Frequency

Now, let's filter out rare words that occur less than a specified number of times:
- We can use the `Counter` class from the `collections` module to count the frequency of each word in the corpus.
- Caution on `alist.sort()` will return `None`.

In [ ]:
# how can we filter the vocab by its frequency?
filtered_vocab = None
# you can use word counter as dictionary
# In python dictionary, dict.keys() gives keys, and dict.values() give values,
# dict.items() give (key, value)

## 7. Filtering Word Pairs

Now that we have our filtered vocabulary, we need to filter our word pairs to only include words that are in our vocabulary:

In [ ]:
# Filter the word_pairs using the vocab
# word_pairs, filtered_vocab
# word_pairs is a list of [word_a, word_b]


In [ ]:
# implement same algorithm with list comprehension


In [ ]:
len(filtered_pairs), len(word_pairs)

## 8. Converting Words to Indices

For efficiency, we'll convert our words to indices according to their position in our vocabulary:

In [ ]:
# convert word into index of vocab
filtered_vocab.index('happily')


This is inefficient because `list.index()` has to scan the list every time. Let's use a dictionary for faster lookups:

In [ ]:
# we can make it faster
# use dictionary to find the index of string


Now, let's convert our word pairs to index pairs more efficiently:

In [ ]:
index_pairs = []

In [ ]:
# Why we don't need idx2tok?

## 9. Creating Initial Word Vectors

Now we'll create random vectors for each word in our vocabulary. These vectors will be adjusted during training:
- We can use `torch.randn` to create random vectors that follow normal distribution.

In [ ]:
# we have to make random vectors for each word in the vocab
# we also have to decide the dimension of the vector

dim = None
vocab_size = None

word_vectors = None

In [ ]:
# what is the vector for harry?


## 10. Understanding Word Relationships with Dot Products

The core of Word2Vec is using dot products to measure relationships between words. Let's explore this concept:

In [ ]:
torch.set_printoptions(sci_mode=False) # Do this to avoid scientific notation


## Dot Product
- Assume we have two vectors $a$ and $b$.
  - $a = [a_1, a_2, a_3, a_4, ..., a_n]$
  - $b = [b_1, b_2, b_3, b_4, ..., b_n]$
- $a \cdot b$ = $\sum _{i=1}^n a_ib_i$  = $a_1b_1 + a_2b_2 + a_3b_3 + a_4b_4 + ... + a_nb_n$

Let's calculate the dot product between "harry" and "potter":


In [ ]:
# calculate P(potter|harry)

dot_product_value_between_potter_harry = None
dot_product_value_between_potter_harry

In [ ]:
# we can get the dot product value for every other words in the vocab
# to get  P(word | harry)
word_dot_dict = {}

word_dot_dict

Now, let's convert these dot products to probabilities using the softmax function:
- We have to convert our prediction into probability distribution to get P(word|harry) so that sum of [P(a|harry), ..., P(potter|harry), ... P(ron|harry), ... ] = 1
- current dot product value is any real number, sometimes called as logit
  - logit from logistic regression. Some values that are not yet converted to 0-1 or value before sigmoid function
  - every probability should be in range (0, 1) (greater than 0, smaller than 1)
  - this can be handled by taking exponential of dot product values, divided by total sum
  - This function is called **Softmax**

- Why we use exponential?
  - Because we want to make every probability in positive range while preserving the order


In [ ]:
from math import exp

word_prob_dict = None

In [ ]:
# Get P(potter|harry)

## 13. Efficient Matrix Operations
![img](https://mkang32.github.io/images/python/khan_academy_matrix_product.png)

Instead of calculating dot products one by one, we can use matrix multiplication for efficiency:


In [ ]:
# get dot product result for every word in the vocabulary

# first, make vector_of_harry into matrix format

# do matrix multiplication


Let's verify that our matrix multiplication gives the same result as individual dot products:

Now let's implement the complete softmax calculation using matrix operations:


In [ ]:
# convert dot product result into exponential

In [ ]:
# get the sum of exponential


In [ ]:
# divide exponential value with sum

## 14. Creating a Probability Function

Let's create a function to calculate probabilities efficiently:

In [ ]:
def get_probs(query_vectors, entire_vectors):
  return None

# get_probs(mat_of_harry, word_vectors)

## 15. Preparing for Training

Before training our Word2Vec model, we need to split our dataset into training and testing sets:

In [ ]:
# Now we can train the word2vec

# Let's think about training pairs
index_pairs # this is our dataset. It's list of list of two integer
# two integer means a pair of neighboring words

# Training set and Test set
# To validate that our model can solve 'unseen' problems
# So we have to split the dataset before training.

# To randomly split the dataset, we will first shuffle the dataset

# random.shuffle(index_pairs) # this will shuffle the list items

In [ ]:
len(train_set), len(test_set)

## 16. Training the Word2Vec Model

Now we'll train our Word2Vec model using batched gradient descent:

In [ ]:
# making batch from train_set
# Batch is a set of training samples, that are calculated together
# And also we update the model after one single batch

## 17. Evaluating the Training

Let's visualize the training loss to see if our model is learning:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_record)

## 18. Testing the Model

Now we'll test our model on the test set:

## 19. Exploring Learned Word Relationships

Let's explore what our model has learned by finding the words most closely related to "harry":

In [ ]:
# P(potter|harry)?
